In [1]:
# ---------------------------------------------------------
# 0. 기본 라이브러리 설치 (Colab 환경용)
#    - pandas, numpy가 안 깔려 있을 수 있어서 한 번 설치
# ---------------------------------------------------------
!pip install pandas numpy

# ---------------------------------------------------------
# 1. 필요한 파이썬 / PyTorch 라이브러리 임포트
# ---------------------------------------------------------
import os
import random
import numpy as np
import pandas as pd
from collections import defaultdict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------
# 2. 실험 재현성을 위한 시드(seed) 고정 함수
#    - 딥러닝 실험은 랜덤 요소가 많아서
#      매번 실행할 때마다 결과가 조금씩 달라질 수 있음
#    - set_seed(42)를 한 번 호출해두면,
#      같은 코드 + 같은 데이터일 때 최대한 비슷한 결과가 나오도록 맞춰줌
# ---------------------------------------------------------
def set_seed(seed=42):
    # 파이썬 내장 random 모듈 시드 고정
    random.seed(seed)
    # NumPy 시드 고정
    np.random.seed(seed)

    # PyTorch (CPU, GPU) 시드 고정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU 사용 시

    # cuDNN 관련 설정
    # - deterministic: 연산을 가능하면 결정론적으로 수행
    # - benchmark: 입력 크기에 따라 최적화 탐색하는 기능인데,
    #              이걸 끄면 속도는 약간 느려질 수 있지만 결과는 더 안정적
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # (선택 사항) CUDA 10.2+ 에서 완전 결정론 모드 설정용 환경변수
    # 일부 연산에서 랜덤성이 남아있을 수 있어서, 이 옵션으로 더 줄여줌
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"


# 위에서 정의한 시드 고정 함수 실제로 한 번 호출
# → 이 셀을 실행한 이후부터는, 가능한 한 같은 결과가 나오도록 랜덤 고정
set_seed(42)


In [2]:
# ---------------------------------------------------------
# 3. MovieLens 100K 데이터셋 다운로드 & 압축 해제
#    - 추천 시스템 실험에서 가장 많이 쓰이는 공개 데이터셋 중 하나
#    - 파일 구조:
#        ml-100k/u.data : 유저-아이템-평점-타임스탬프
#        ml-100k/u.item : 아이템(영화) 메타데이터 (제목, 장르 등)
# ---------------------------------------------------------

DATA_DIR = "./ml-100k"  # 데이터셋을 저장할 로컬 폴더 경로

# 폴더가 없으면, 아직 데이터셋이 없는 것으로 판단하고 다운로드 + 압축 해제
if not os.path.exists(DATA_DIR):
    # MovieLens 100K zip 파일 다운로드
    !wget -nc https://files.grouplens.org/datasets/movielens/ml-100k.zip
    # 조용히(unzip -qq) 압축 해제 → ./ml-100k 폴더가 생성됨
    !unzip -qq ml-100k.zip


--2025-11-23 05:19:01--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  2.75MB/s    in 1.7s    

2025-11-23 05:19:04 (2.75 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [9]:
# ---------------------------------------------------------
# 4. MovieLens 평점 데이터 로드 & 전처리
#    - u.data 파일: user_id, item_id, rating, timestamp (탭으로 구분)
#    - SASRec은 "시퀀스(시간 순서) 기반 추천"을 하므로,
#      어떤 유저가 어떤 아이템을 어떤 순서로 봤는지가 중요함.
#      그 준비를 이 셀에서 시작한다고 보면 됨.
# ---------------------------------------------------------

ratings_path = os.path.join(DATA_DIR, "u.data")

# u.data 컬럼 정보:
#   user_id   : 유저 번호 (원본 id, 1 ~ 943 사이 등)
#   item_id   : 영화 번호 (원본 id, 1 ~ 1682 사이 등)
#   rating    : 1~5 점수
#   timestamp : 시청 시점 (Unix time 형태)
df = pd.read_csv(
    ratings_path,
    sep="\t",
    names=["user", "item", "rating", "timestamp"]
)

# ---------------------------------------------------------
# 4-1. 암묵적 피드백(implicit feedback)으로 변환
#      - 기존: 1~5점 "선호 정도"가 있는 명시적 피드백(explicit)
#      - 지금은: "봤다 / 안 봤다"에 가까운 implicit 데이터로 사용
#      - 보통 rating >= 3 (3, 4, 5점)을 '좋게 본 아이템'으로 간주
# ---------------------------------------------------------
df = df[df["rating"] >= 3] # 힌트 : rating이 3점 이상인 것만 인덱싱

# ---------------------------------------------------------
# 4-2. user, item id를 0부터 시작하는 연속 id로 다시 매핑
#      - 원래 MovieLens의 user_id, item_id는 중간에 숫자가 비거나
#        범위가 넓을 수 있음 (sparse한 id)
#      - 딥러닝 embedding layer는 index가 0~N-1 형태로 이어지는 게 편하므로,
#        "내부용 연속 id"로 바꿔줌.
#      - 여기서는 0을 padding 전용으로 예약하기 위해
#        실제 유저/아이템은 1부터 할당.
# ---------------------------------------------------------

# 원본 user id를 정렬해서, 1부터 차례대로 새 id를 부여
# ex) 원래 user id가 [5,10,70]이라면 → 내부 id [1,2,3]
user2id = {
    u: i + 1
    for i, u in enumerate(sorted(df["user"].unique())) # 힌트 : 데이터에 포함된 고유 값
}  # 0은 padding용으로 비워둠

# 원본 item id에 대해서도 동일하게 연속 id 부여
item2id = {
    i: j + 1
    for j, i in enumerate(sorted(df["item"].unique()))
}  # 0은 padding용으로 비워둠

# DataFrame에 새 id를 실제로 매핑
df["user"] = df["user"].map(user2id)
df["item"] = df["item"].map(item2id)

# 최종 유저/아이템 수 확인
n_users = df["user"].nunique() # 힌트 : 데이터에 포함된 고유 값의 개수
n_items = df["item"].nunique()
print(f"#users = {n_users}, #items = {n_items}")

#users = 943, #items = 1574


In [11]:
df

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
5,298,469,4,884182806
7,253,460,5,891628467
8,305,446,3,886324817
...,...,...,...,...
99992,721,262,3,877137285
99994,378,78,3,880056976
99995,880,471,3,880175444
99996,716,204,5,879795543


In [15]:
# ---------------------------------------------------------
# 5. 유저별 시퀀스 만들기 + train/val/test 분할
#    SASRec은 "시퀀스(순서) 기반 추천" 모델이라,
#    각 유저가 시간을 따라 어떤 아이템을 봤는지 순서대로 정리해주는 단계입니다.
# ---------------------------------------------------------

# 5-1. 유저-타임스탬프 기준으로 정렬
#   - 같은 유저 안에서 timestamp 오름차순으로 정렬해서
#     "시청한 순서"를 복원합니다.
df = df.sort_values(["user", "timestamp"]) # 힌트 : 오름차순 정렬

# 5-2. 유저별로 시청한 item id를 순서대로 모아 시퀀스 생성
#   - user_sequences[u] = [i1, i2, i3, ...] 형태
#   - 여기서 각 원소는 우리가 앞에서 만든 "내부 아이템 id"입니다.
user_sequences = defaultdict(list)
for row in df.itertuples():
    user_sequences[row.user].append(row.item)

# 5-3. 너무 짧은 시퀀스를 가진 유저는 제거
#   - 예를 들어, 어떤 유저가 영화 1~2개만 봤다면
#     "과거를 보고 미래를 예측"하는 시퀀스 모델 입장에서 정보가 거의 없음.
#   - 여기서는 최소 5개 이상 본 유저만 사용하도록 필터링.
MIN_LEN = 5
filtered_user_sequences = {
    u: seq for u, seq in user_sequences.items() if len(seq) >= MIN_LEN
}
user_sequences = filtered_user_sequences
print("After filtering, #users:", len(user_sequences))

# ---------------------------------------------------------
# 5-4. train / val / test로 시퀀스를 나누기
#
#   아이디어:
#   - 한 유저의 전체 시청 시퀀스: [i1, i2, ..., i_{n-2}, i_{n-1}, i_n]
#   - train:   i1 ~ i_{n-2} 까지 (앞부분 모두)
#   - val:     i_{n-1} (마지막에서 두 번째 아이템)
#   - test:    i_n     (마지막 아이템)
#
#   이렇게 나누면:
#   - 학습할 때는 과거 시퀀스들을 사용해서 "다음 아이템"을 맞추도록 학습하고,
#   - 검증/테스트할 때는, 유저의 최근 기록을 입력으로 넣고
#     마지막/마지막 전 아이템을 잘 맞추는지 평가할 수 있음.
# ---------------------------------------------------------

user_train = dict()  # 각 유저의 학습용 시퀀스 (리스트)
user_val = dict()    # 검증(Validation)용 타깃 아이템 (int 혹은 None)
user_test = dict()   # 테스트(Test)용 타깃 아이템 (int)

for u, seq in user_sequences.items():
    if len(seq) < 2:
        # (이론상 여기까지 오면 거의 없겠지만)
        # 길이가 2 미만이면 "과거 → 미래" 구조를 만들 수 없으므로 스킵
        continue

    if len(seq) >= 3:
        # 길이가 3 이상이면:
        #   [i1, i2, ..., i_{n-2}, i_{n-1}, i_n]
        #   - train 시퀀스  : i1 ~ i_{n-2}
        #   - val target   : i_{n-1}
        #   - test target  : i_n
        user_train[u] = seq[:-2] # 힌트 : 가장 최근 두 아이템을 제외한 (앞부분) 전체 시퀀스를 가져오는 인덱싱
        user_val[u] = seq[-2] # 힌트 : 가장 최근에서 두 번째 아이템 (Validation Target)을 가져오는 인덱싱
        user_test[u] = seq[-1] # 힌트 : 가장 최근 아이템 (Test Target)을 가져오는 인덱싱
    else:
        # 길이가 정확히 2인 경우: [i1, i2]
        #   - train: [i1]
        #   - val  : 없음 (None 처리)
        #   - test : i2
        #   → 그래도 최소한 test 평가에는 쓸 수 있게 구성
        user_train[u] = seq[:-1]   # [i1]
        user_val[u] = None
        user_test[u] = seq[-1]     # i2

print(f"#users with train seq = {len(user_train)}")

After filtering, #users: 943
#users with train seq = 943


In [16]:
# ---------------------------------------------------------
# 6. SASRec 학습용 Dataset 정의
#    - 각 유저의 시청 시퀀스를 가지고
#      "prefix → 다음 아이템" 형태의 학습 샘플을 여러 개 만들어냄.
#    - 예: [i1, i2, i3, i4] 라는 시퀀스가 있으면
#         ( [i1]         → i2 )
#         ( [i1, i2]     → i3 )
#         ( [i1, i2, i3] → i4 )
#      이런 식으로 여러 훈련 샘플을 생성.
# ---------------------------------------------------------

MAX_SEQ_LEN = 50  # 모델이 한 번에 볼 수 있는 최대 시퀀스 길이
                   # (뒤에서부터 최대 50개까지만 사용, 나머지는 잘라냄)

class SASRecTrainDataset(Dataset):
    """
    SASRec 학습용 PyTorch Dataset.
    - 입력: user_train_dict[u] = 그 유저의 학습용 시퀀스 (아이템 id 리스트)
    - 출력:
      - seq  : 길이 max_len인 시퀀스 (왼쪽은 padding, 오른쪽은 실제 아이템)
      - target: seq 다음에 나와야 할 정답 아이템 id
    """
    def __init__(self, user_train_dict, max_len):
        self.max_len = max_len
        self.seqs = []     # 모든 학습 샘플의 입력 시퀀스를 모아둘 리스트
        self.targets = []  # 각 시퀀스에 대한 다음 아이템(정답 라벨)

        # user_train_dict: {user_id: [i1, i2, ..., iM]}
        for u, items in user_train_dict.items():
            if len(items) < 2:
                # 길이가 1인 유저는 "과거 → 다음" 구조의 샘플을 만들 수 없으므로 패스
                continue

            # items = [i1, i2, ..., iM]
            # t = 1 .. M-1 에 대해
            #   prefix: [i1, ..., i_t]
            #   target: i_{t+1}
            for t in range(1, len(items)):
                target = items[t]   # 힌트 : 현재 시점의 아이템을 "예측해야 할 정답"으로 사용
                seq = items[:t]     # 힌트 : 그 이전까지의 prefix [i1, ..., i_{t}]

                # (1) 너무 길면 뒤에서부터 max_len개만 남기고 앞부분은 버림
                #     - SASRec은 '최근 행동'에 더 집중하는 모델이라,
                #       과거가 길어질수록 가장 최근 max_len개만 보는 게 일반적.
                if len(seq) > max_len:
                    seq = seq[-max_len:]

                # (2) 왼쪽 padding
                #     - embedding + transformer 입력으로 넣기 위해
                #       길이를 전부 max_len으로 맞춰야 함.
                #     - 0은 "아무 것도 없음"을 나타내는 padding id.
                pad_len = max_len - len(seq)
                seq = [0] * pad_len + seq

                # 리스트에 추가
                self.seqs.append(seq)
                self.targets.append(target)

        # 최종적으로는 Tensor 형태로 바꿔서 저장
        self.seqs = torch.LongTensor(self.seqs)       # (N, max_len) # 힌트 : 무슨 형태로 바꾼다고 했죠?
        self.targets = torch.LongTensor(self.targets) # (N,)

    def __len__(self):
        # 전체 학습 샘플 개수
        return len(self.targets)

    def __getitem__(self, idx):
        # idx번째 샘플의 (입력 시퀀스, 정답 아이템) 반환
        return self.seqs[idx], self.targets[idx]


# 위에서 정의한 Dataset을 실제로 생성
train_dataset = SASRecTrainDataset(user_train, MAX_SEQ_LEN)
print("Train samples:", len(train_dataset))

Train samples: 79691


In [21]:
# ---------------------------------------------------------
# 7. SASRec 모델 정의
#    - Self-Attentive Sequential Recommendation (SASRec)
#    - "유저가 최근에 본 아이템들의 시퀀스"를 입력으로 받아
#      "다음에 볼 아이템"에 대한 점수(logits)를 출력하는 모델.
#    - 핵심 아이디어:
#      1) 아이템 임베딩 + 위치(positional) 임베딩
#      2) Transformer Encoder (Self-Attention)로 시퀀스를 인코딩
#      3) 마지막 시점(hidden state)만 뽑아서 다음 아이템을 예측
# ---------------------------------------------------------

class SASRec(nn.Module):
    def __init__(self, n_items, d_model=64, n_layers=2, n_heads=2, max_len=50, dropout=0.2):
        """
        n_items : 전체 아이템 개수
        d_model : 임베딩 차원 (= Transformer hidden 차원)
        n_layers: Transformer encoder layer 개수
        n_heads : Multi-head attention의 head 개수
        max_len : 한 시퀀스의 최대 길이 (입력 L)
        dropout : 드롭아웃 비율
        """
        super().__init__()
        self.n_items = n_items
        self.max_len = max_len
        self.d_model = d_model

        # -------------------------------------------------
        # 1) 아이템 임베딩 + 포지션 임베딩
        # -------------------------------------------------

        # 아이템 임베딩
        # - 각 item id를 d_model 차원의 벡터로 매핑
        # - 0번 id는 padding용이므로, n_items + 1 크기로 만듦
        # - padding_idx=0: 이 위치에는 gradient가 전파되지 않음
        self.item_emb = nn.Embedding(n_items + 1, d_model, padding_idx=0)

        # 위치(positional) 임베딩
        # - 시퀀스 내 "순서" 정보를 넣기 위해 사용 (0 ~ max_len-1)
        self.pos_emb = nn.Embedding(max_len, d_model)

        # -------------------------------------------------
        # 2) Transformer Encoder
        # -------------------------------------------------
        # - SASRec는 기본적으로 "TransformerEncoderLayer" 구조를 사용
        # - 각 위치의 아이템이, 같은 시퀀스 내 다른 위치의 아이템을 Attention으로 바라보며
        #   어떤 아이템들이 현재 시점에 중요한 영향을 주는지 학습
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,              # hidden 차원
            nhead=n_heads,                # multi-head attention head 수
            dim_feedforward=4 * d_model,  # FFN 내부 차원 (일반적으로 4배 정도 사용)
            dropout=dropout,
            batch_first=True              # 입력 shape을 (B, L, D)로 받겠다는 옵션
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=n_layers           # Encoder layer를 몇 층 쌓을지
        )

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)

        # -------------------------------------------------
        # 3) 출력 레이어 (아이템 space로 projection)
        # -------------------------------------------------
        # - 마지막 hidden state를 "각 아이템에 대한 점수(logit)"로 바꾸는 선형 레이어
        # - weight tying: item embedding weight와 output weight를 공유
        #   → 파라미터 수 감소 + 성능 이점이 알려져 있음
        self.output_layer = nn.Linear(d_model, n_items + 1, bias=False)
        self.output_layer.weight = self.item_emb.weight  # weight tying

        # 파라미터 초기화
        self._reset_parameters()

    def _reset_parameters(self):
        """
        임베딩 파라미터들을 정규분포로 초기화.
        (Transformer 기본 구현에서도 비슷한 초기화를 자주 사용)
        """
        nn.init.normal_(self.item_emb.weight, mean=0, std=0.01)
        nn.init.normal_(self.pos_emb.weight, mean=0, std=0.01)

    def forward(self, seq):
        """
        seq: (B, L) 형태의 LongTensor
             - 각 원소는 item id (0은 padding)
             - 왼쪽 padding, 오른쪽에 실제 아이템들이 쌓여 있는 형태
        return:
            h: (B, L, d_model)
               - self-attention을 통과한 각 위치의 hidden state
        """
        device = seq.device
        B, L = seq.shape

        # 1) 위치 인덱스 만들기: [0, 1, 2, ..., L-1]
        #    - 각 batch에 대해 동일한 위치 인덱스를 사용
        pos_idx = torch.arange(L, dtype=torch.long, device=device)
        pos_idx = pos_idx.unsqueeze(0).expand(B, L)  # (B, L)

        # 2) 아이템 임베딩 + 위치 임베딩
        #    - "이 시점에 어떤 아이템을 봤고, 그것이 시퀀스에서 몇 번째인지" 정보를 합침
        x = self.item_emb(seq) + self.pos_emb(pos_idx)  # (B, L, d_model)

        # LayerNorm + Dropout
        x = self.layer_norm(x)
        x = self.dropout(x)

        # 3) 마스크 생성

        # (a) padding mask: seq == 0 인 위치는 실제 아이템이 아니므로 attention에서 무시
        #     - shape: (B, L), True인 곳은 무시
        pad_mask = (seq == 0)

        # (b) causal mask (상삼각 행렬)
        #     - 현재 시점 t가, 미래 시점(t+1, t+2, ...)을 보지 못하도록 막는 역할
        #     - TransformerEncoder의 mask 인자는 (L, L) 형태.
        #     - True인 위치는 attention에서 무시됨.
        attn_mask = torch.triu(
            torch.ones(L, L, device=device),
            diagonal=1
        ).bool()  # 상삼각 (i < j) 위치가 True

        # 4) Transformer Encoder 통과
        #    - 각 위치의 hidden state는
        #      자신의 과거 위치들(및 자기 자신)을 self-attention으로 요약한 표현이 됨.
        h = self.encoder(
            x,
            mask=attn_mask,                 # 시계열 방향을 지키기 위한 causal mask
            src_key_padding_mask=pad_mask   # padding 위치 무시
        )  # 출력: (B, L, d_model)

        return h

    def predict_next(self, seq):
        """
        seq: (B, L) 입력 시퀀스
        return:
            logits: (B, n_items+1)
                    - 각 배치에 대해 "다음 아이템이 각 item일 점수(logit)"
        """
        # 1) 전체 시퀀스에 대한 hidden state 계산
        h = self.forward(seq)  # (B, L, d_model)

        # 2) 우리가 관심 있는 건 "마지막 위치"의 표현
        #    - 입력 시퀀스 기준으로 가장 최신 시점(L-1)의 hidden state
        h_last = h[:, -1, :]  # (B, d_model)

        # 3) 마지막 hidden state를 아이템 space로 투사
        #    - 각 아이템에 대해 점수(로짓) 계산
        logits = self.output_layer(h_last)  # (B, n_items+1)

        return logits

In [24]:
# ---------------------------------------------------------
# 8. SASRec 평가 함수 (Recall@K, NDCG@K)
#    - 학습된 SASRec 모델이 "다음 아이템"을 얼마나 잘 맞추는지 측정.
#    - 유저별로:
#        입력: train 시퀀스 (user_train[u])
#        정답: user_eval_target[u]  (val 또는 test에서 하나의 아이템)
#      를 사용해서 Top-K 추천 성능을 계산합니다.
# ---------------------------------------------------------

def evaluate_sasrec(
    model,
    user_train,        # {user_id: [i1, i2, ..., iM]}  학습에 사용한 prefix 시퀀스
    user_eval_target,  # {user_id: target_item}       평가용 정답 아이템 (val or test)
    max_len,
    n_items,
    mode="val",        # 출력 로그에 붙일 태그 (예: "val", "test")
    k_list=[20],       # 평가할 K 리스트 (예: [5, 10, 20] 등)
    device="cuda"
):
    model.eval()       # 평가 모드 (Dropout, BatchNorm 등 비활성화)
    recalls = {k: [] for k in k_list}  # 각 K에 대한 Recall 값을 저장할 리스트
    ndcgs = {k: [] for k in k_list}    # 각 K에 대한 NDCG 값을 저장할 리스트

    with torch.no_grad():  # 평가에서는 gradient 계산 불필요
        # 유저별로 반복
        for u, train_items in user_train.items():
            # 이 유저의 평가 대상(target) 아이템 (val 또는 test에서 하나)
            target = user_eval_target.get(u, None)
            if target is None:
                # 이 유저에 대해 평가할 타깃이 없으면 스킵
                continue

            # -------------------------
            # 1) 이 유저의 입력 시퀀스 만들기
            # -------------------------
            seq = train_items.copy()
            if len(seq) == 0:
                continue  # 비어있으면 스킵

            # (a) 시퀀스가 너무 길면 최근 max_len개만 사용
            if len(seq) > max_len:
                seq = seq[-max_len:]

            # (b) 왼쪽 padding으로 길이를 max_len에 맞추기
            pad_len = max_len - len(seq)
            seq = [0] * pad_len + seq  # 0은 padding id

            # (c) Tensor로 변환 (배치 차원 B=1 추가)
            seq_tensor = torch.LongTensor(seq).unsqueeze(0).to(device)  # (1, L)

            # -------------------------
            # 2) 모델로부터 다음 아이템 점수(logits) 얻기
            # -------------------------
            logits = model.predict_next(seq_tensor)  # (1, n_items+1)
            scores = logits.squeeze(0).cpu().numpy()  # (n_items+1,)

            # -------------------------
            # 3) 이미 본 아이템은 추천 후보에서 제외
            # -------------------------
            #   - scores[0] : padding id → 매우 작은 값으로 설정
            scores[0] = -1e9
            #   - 이 유저가 train에서 이미 본 아이템들도 추천에서 제외
            seen = set(train_items)
            for item in seen:
                scores[item] = -1e9

            # -------------------------
            # 4) 점수 내림차순 정렬하여 랭킹 계산
            # -------------------------
            ranking = np.argsort(-scores)  # 점수가 큰 순서대로 인덱스 정렬
            # target 아이템이 ranking에서 몇 번째 위치인지 찾기
            rank = np.where(ranking == target)[0]
            if len(rank) == 0:
                # (이론상 거의 없겠지만) target이 랭킹에 없으면 스킵
                continue
            rank = rank[0]  # 실제 순위 (0-based index)

            # -------------------------
            # 5) 각 K에 대해 Recall@K, NDCG@K 계산
            # -------------------------
            for k in k_list:
                if rank < k:
                    # Top-K 안에 정답이 들어 있으면
                    #   Recall@K: 1
                    #   NDCG@K  : 1 / log2(rank+2)
                    #             (순위가 높을수록 더 큰 점수)
                    recalls[k].append(1.0)
                    ndcgs[k].append(1.0 / np.log2(rank + 2))
                else:
                    # Top-K 안에 정답이 없으면
                    recalls[k].append(0.0)
                    ndcgs[k].append(0.0)

    # -------------------------
    # 6) 유저별 결과를 평균 내어 최종 지표 계산
    # -------------------------
    result = {}
    for k in k_list:
        if len(recalls[k]) == 0:
            # 평가 대상 유저가 없었다면 0으로 처리
            result[f"Recall@{k}"] = 0.0
            result[f"NDCG@{k}"] = 0.0
        else:
            result[f"Recall@{k}"] = float(np.mean(recalls[k]))
            result[f"NDCG@{k}"] = float(np.mean(ndcgs[k]))

    # 예: [val] Recall@20: 0.12 NDCG@20: 0.05 형태로 출력
    print(f"[{mode}] ", end="")
    print(" ".join([f"{m}: {v:.4f}" for m, v in result.items()]))

    return result


In [25]:
# ---------------------------------------------------------
# 9. 학습 환경 설정 + DataLoader + 학습 루프
#    - GPU/CPU 선택
#    - DataLoader로 배치 구성
#    - SASRec 모델 생성, 손실함수/옵티마이저 정의
#    - 에폭을 돌면서 학습 + 검증(Recall@20) + best 모델 저장
# ---------------------------------------------------------

# 9-1. 사용할 디바이스 선택 (가능하면 GPU, 아니면 CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 9-2. 하이퍼파라미터 설정
BATCH_SIZE = 256   # 한 번에 학습할 샘플 수
EPOCHS = 50        # 전체 데이터셋을 몇 번 반복 학습할지
LR = 5e-4          # 학습률 (learning rate)

# 9-3. DataLoader용 랜덤 시드 고정
#  - DataLoader 내에서 셔플(shuffle=True)을 할 때도
#    랜덤 시퀀스가 재현되도록 generator에 시드를 따로 고정
g = torch.Generator()
g.manual_seed(42)

# 9-4. 학습용 DataLoader 정의
#  - train_dataset에서 BATCH_SIZE 단위로 데이터를 꺼내와서
#    (seq_batch, target_batch) 형태로 모델에 전달
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,    # 매 epoch마다 데이터 순서를 섞어서 학습
    drop_last=False, # 마지막 배치가 BATCH_SIZE보다 작아도 그대로 사용
    generator=g,     # ★ 랜덤 시드가 고정된 generator 사용
    num_workers=0    # 결정론적인 실행을 위해 0으로 설정 (멀티프로세스 X)
)

# 9-5. SASRec 모델 생성
model = SASRec(
    n_items=n_items,       # 아이템 개수
    d_model=128,           # 임베딩/히든 차원
    n_layers=3,            # Transformer encoder layer 3층
    n_heads=4,             # multi-head attention head 4개
    max_len=MAX_SEQ_LEN,   # 한 시퀀스 최대 길이 (앞에서 50으로 정의)
    dropout=0.3            # 드롭아웃 비율
).to(device)               # 선택된 디바이스(GPU/CPU)로 올리기

# 9-6. 손실 함수(크로스 엔트로피) 정의
#  - logits: (B, n_items+1)  → 각 아이템에 대한 점수
#  - target: (B,)            → 정답 item id (1 ~ n_items, 0은 padding이라 안 씀)
criterion = nn.CrossEntropyLoss()

# 9-7. 옵티마이저 정의 (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# 9-8. 검증 성능 기준으로 best 모델을 저장하기 위한 변수들
best_val_recall20 = 0.0
best_state_dict = None

# ---------------------------------------------------------
# 9-9. 학습 루프
#    - 각 epoch마다:
#      1) train_loader로 한 바퀴 돌면서 모델 파라미터 업데이트
#      2) evaluate_sasrec으로 val Recall@20 / NDCG@20 측정
#      3) 지금 모델이 가장 좋으면(best) weight를 저장
# ---------------------------------------------------------

for epoch in range(1, EPOCHS + 1):
    model.train()      # 학습 모드 (Dropout 등 활성화)
    total_loss = 0.0   # 에폭 동안의 총 손실 누적

    # ① 한 epoch 동안 train_loader를 모두 순회
    for seq_batch, target_batch in train_loader:
        # 배치 데이터를 디바이스로 이동
        seq_batch = seq_batch.to(device)         # (B, L)
        target_batch = target_batch.to(device)   # (B,)

        # 모델이 예측한 다음 아이템 logits
        logits = model.predict_next(seq_batch)   # (B, n_items+1)

        # 크로스 엔트로피 손실 계산
        loss = criterion(logits, target_batch)

        # 기울기 초기화 → 역전파 → 파라미터 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 배치 손실을 전체 데이터 크기에 맞춰 누적
        total_loss += loss.item() * seq_batch.size(0)

    # ② epoch이 끝난 후 평균 Train Loss 출력
    avg_loss = total_loss / len(train_dataset)
    print(f"[Epoch {epoch}] Train Loss: {avg_loss:.4f}")

    # ③ 검증(Validation) 성능 측정
    #    - 여기서는 Recall@20, NDCG@20만 사용
    val_result = evaluate_sasrec(
        model,
        user_train=user_train,         # 입력용 prefix 시퀀스
        user_eval_target=user_val,     # 검증용 target 아이템
        max_len=MAX_SEQ_LEN,
        n_items=n_items,
        mode="val",
        k_list=[20],                   # ★ R@20 / NDCG@20만 계산
        device=device
    )

    # ④ Recall@20 기준으로 best 모델 갱신 여부 확인
    if val_result["Recall@20"] > best_val_recall20:
        best_val_recall20 = val_result["Recall@20"]
        # best weight는 CPU 텐서로 따로 저장해 둔다 (나중에 로드하기 쉽게)
        best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
        print("  ** New best model (by Recall@20 on val) **")


Device: cuda
[Epoch 1] Train Loss: 6.3730
[val] Recall@20: 0.1029 NDCG@20: 0.0405
  ** New best model (by Recall@20 on val) **
[Epoch 2] Train Loss: 5.8614
[val] Recall@20: 0.1082 NDCG@20: 0.0476
  ** New best model (by Recall@20 on val) **
[Epoch 3] Train Loss: 5.7602
[val] Recall@20: 0.1135 NDCG@20: 0.0450
  ** New best model (by Recall@20 on val) **
[Epoch 4] Train Loss: 5.6968
[val] Recall@20: 0.1241 NDCG@20: 0.0549
  ** New best model (by Recall@20 on val) **
[Epoch 5] Train Loss: 5.6412
[val] Recall@20: 0.1400 NDCG@20: 0.0593
  ** New best model (by Recall@20 on val) **
[Epoch 6] Train Loss: 5.5913
[val] Recall@20: 0.1315 NDCG@20: 0.0518
[Epoch 7] Train Loss: 5.5497
[val] Recall@20: 0.1421 NDCG@20: 0.0569
  ** New best model (by Recall@20 on val) **
[Epoch 8] Train Loss: 5.5120
[val] Recall@20: 0.1410 NDCG@20: 0.0576
[Epoch 9] Train Loss: 5.4822
[val] Recall@20: 0.1357 NDCG@20: 0.0552
[Epoch 10] Train Loss: 5.4530
[val] Recall@20: 0.1442 NDCG@20: 0.0572
  ** New best model (by Re

In [26]:
# ---------------------------------------------------------
# 10. Best 모델 로드 후 Test 평가
#    - 학습 과정에서 가장 높은 Recall@20 (on val)을 기록한
#      모델 상태(best_state_dict)를 실제 model에 불러와서
#      최종적으로 test 세트에서 성능을 측정하는 단계입니다.
# ---------------------------------------------------------

# 10-1. 학습 중에 저장해 둔 best_state_dict가 있다면 로드
if best_state_dict is not None:
    # model에 best weight 로드
    model.load_state_dict(best_state_dict)
    model.to(device)  # 혹시 모를 디바이스 mismatch 방지
    print("Best model loaded for test evaluation.")

# 10-2. Test 세트에서 최종 성능 평가
#    - user_train: 입력용 prefix 시퀀스 (train 부분)
#    - user_test : 각 유저에 대해 "마지막 아이템"을 target으로 사용
test_result = evaluate_sasrec(
    model,
    user_train=user_train,
    user_eval_target=user_test,  # ★ test 타깃
    max_len=MAX_SEQ_LEN,
    n_items=n_items,
    mode="test",                 # 출력 로그에 [test] 표시
    k_list=[20],                 # Recall@20, NDCG@20 계산
    device=device
)


Best model loaded for test evaluation.
[test] Recall@20: 0.1220 NDCG@20: 0.0503


In [28]:
# ---------------------------------------------------------
# 11. 아이템 ID → 영화 제목 매핑 만들기
#    - 지금까지 모델은 "내부 아이템 id(숫자)"만 알고 있음.
#    - 사람 눈으로 결과를 이해하려면,
#      이 id를 실제 "영화 제목(title)"로 바꿔 보여줄 필요가 있음.
#    - 이 셀에서는:
#      1) 원본 MovieLens 평점 데이터(u.data)를 다시 읽고
#      2) raw item id ↔ 내부 id 매핑을 만들고
#      3) u.item에서 영화 제목을 읽어와
#      4) 최종적으로 "내부 id → 제목" 딕셔너리(id2title)를 만듦.
# ---------------------------------------------------------

# 11-1. 원본 평점 데이터 다시 로드 (rating, timestamp까지 포함)
raw_ratings = pd.read_csv(
    os.path.join(DATA_DIR, "u.data"),
    sep="\t",
    names=["user", "item_raw", "rating", "timestamp"]
)

# 11-2. 우리가 사용할 아이템 집합 정의
#   - 여기서는 "어느 정도 이상 좋아한 영화"만 아이템 후보로 쓰기 위해
#     rating >= 3 인 영화만 사용 (기준은 실험 설계에 따라 바꿀 수 있음)
raw_ratings = raw_ratings[raw_ratings["rating"] >= 3]

# 11-3. raw item id → inner id 매핑 구성
#   - raw item id: MovieLens 원본에서의 영화 번호 (1~1682 등)
#   - inner id   : 우리가 모델에서 사용하는 "연속적인 아이템 id" (1~N, 0은 padding)
#   - 정렬된 raw item id 순서대로 1부터 번호를 매겨줌.
raw_item_ids = sorted(raw_ratings["item_raw"].unique())
item2id = {raw_i: j + 1 for j, raw_i in enumerate(raw_item_ids)}

# inner id → raw item id 역매핑도 함께 만들어둠
id2item_raw = {v: k for k, v in item2id.items()}

# 11-4. u.item 파일에서 영화 메타데이터(제목 등) 로드
#   - 컬럼 구조:
#     0: item_raw (원본 영화 id)
#     1: title (영화 제목)
#     2: release_date
#     3: video_release_date
#     4: imdb_url
#     5~: 장르 one-hot (Action, Comedy, Drama, ...)
items_path = os.path.join(DATA_DIR, "u.item")
item_meta = pd.read_csv(
    items_path,
    sep="|",
    encoding="latin-1",
    header=None,
    names=[
        "item_raw", "title", "release_date", "video_release", "imdb_url",
        "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime",
        "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
        "Romance", "Sci-Fi", "Thriller", "War", "Western"
    ]
)

# raw item id → 제목(title) 매핑
rawid2title = dict(zip(item_meta["item_raw"], item_meta["title"]))

# 11-5. 최종: inner id → 제목(title) 매핑 만들기
#   - 모델이 사용하는 inner id를 사람이 읽을 수 있는 영화 제목으로 바꾸기 위함.
id2title = {
    inner: rawid2title[raw]
    for inner, raw in id2item_raw.items()
}


In [29]:
# ---------------------------------------------------------
# 12. 특정 유저에 대한 SASRec 추천 & 결과 출력 함수
#    - sasrec_recommend_for_user:
#        → "user_id" 하나를 입력으로 받아,
#           그 유저에게 추천할 Top-K 아이템 id 리스트를 반환.
#    - show_user_history_and_recs:
#        → "최근에 본 영화 제목들"과
#           "SASRec가 추천한 영화 제목들"을 나란히 보여줌.
#    - 이 두 함수를 이용하면,
#      숫자 id가 아니라 실제 영화 제목 기준으로
#      SASRec의 추천 결과를 눈으로 확인할 수 있음.
# ---------------------------------------------------------

def sasrec_recommend_for_user(
    model,
    user_id,
    user_train,
    topk=10,
    max_len=MAX_SEQ_LEN,
    device="cuda"
):
    """
    특정 user_id에 대해 SASRec 추천 결과 Top-K를 반환하는 함수.

    입력:
      - model      : 학습된 SASRec 모델
      - user_id    : 추천을 보고 싶은 유저의 id (내부 user id)
      - user_train : {user_id: [i1, i2, ..., iM]} 형태의 학습 시퀀스 딕셔너리
      - topk       : 몇 개의 아이템을 추천할지 (기본 10개)
      - max_len    : 모델 입력 시퀀스 최대 길이
      - device     : "cuda" 또는 "cpu"

    출력:
      - top_items: 길이 topk인 numpy 배열
                   (추천된 아이템의 "내부 아이템 id"가 들어 있음)
    """
    model.eval()  # 평가 모드 (Dropout 등 비활성화)

    with torch.no_grad():  # 추천 시에는 gradient 계산이 필요 없음
        # 1) 이 유저의 train 시퀀스를 가져와서 복사
        seq = user_train[user_id].copy()

        # 2) max_len보다 길면 최근 max_len개만 남기기
        #    - SASRec은 "최근 행동"에 더 집중하므로 뒤에서 자르는 게 맞음
        if len(seq) > max_len:
            seq = seq[-max_len:]

        # 3) 왼쪽 padding으로 길이를 max_len에 맞추기
        pad_len = max_len - len(seq)
        seq = [0] * pad_len + seq

        # 4) Tensor로 변환 (배치 차원 B=1 추가)
        seq_tensor = torch.LongTensor(seq).unsqueeze(0).to(device)  # (1, L)

        # 5) 모델에 넣어서 "다음 아이템"에 대한 점수 얻기
        logits = model.predict_next(seq_tensor)   # (1, n_items+1)
        scores = logits.squeeze(0).cpu().numpy()  # (n_items+1,)

        # 6) 추천에서 제외해야 할 것들 처리
        #    (a) 0번: padding id → 아주 작은 점수 부여
        scores[0] = -1e9

        #    (b) 이미 사용자가 본 아이템들 → 다시 추천하지 않도록 제외
        seen = set(user_train[user_id])
        for item in seen:
            scores[item] = -1e9

        # 7) 점수를 기준으로 내림차순 정렬 → 상위 topk개 선택
        ranking = np.argsort(-scores)   # 큰 점수 순으로 인덱스 정렬
        top_items = ranking[:topk]      # 상위 topk개 아이템 id

    return top_items


def show_user_history_and_recs(user_id, topk=10, recent_n=10):
    """
    특정 유저에 대해:
      1) 최근에 본 영화 N개를 제목으로 보여주고
      2) SASRec 추천 Top-K 영화 제목도 같이 출력하는 함수.

    - 입력:
      - user_id : 보고 싶은 유저 id
      - topk    : 추천 상위 몇 개를 보여줄지
      - recent_n: 최근 시청 기록 몇 개를 보여줄지
    """

    # -------------------------
    # 1) 최근 시청 영화 N개의 제목 뽑기
    # -------------------------
    seq = user_train[user_id]              # 이 유저의 전체 train 시퀀스
    recent_seq = seq[-recent_n:]           # 뒤에서 recent_n개만 뽑기
    recent_titles = [
        id2title.get(i, f"item_{i}")
        for i in recent_seq
    ]

    # -------------------------
    # 2) SASRec 추천 결과 가져오기 (Top-K)
    # -------------------------
    top_items = sasrec_recommend_for_user(
        model,
        user_id,
        user_train,
        topk=topk
    )
    rec_titles = [
        id2title.get(i, f"item_{i}")
        for i in top_items
    ]

    # -------------------------
    # 3) 보기 좋게 DataFrame으로 정리
    # -------------------------
    # 최근 시청 목록:
    #   - order: 1이 가장 오래된 게 아니라,
    #            "최근순"을 표현하기 위해 거꾸로 번호를 매김
    hist_df = pd.DataFrame({
        "order": list(range(len(recent_titles), 0, -1)),
        "recent_watched": list(reversed(recent_titles))  # 가장 최근이 맨 위로 오게
    })

    # 추천 목록:
    #   - rank: 1위 추천, 2위 추천, ... topk위 추천
    rec_df = pd.DataFrame({
        "rank": list(range(1, topk + 1)),
        "recommended": rec_titles
    })

    # -------------------------
    # 4) Colab에서 표 형태로 출력
    # -------------------------
    from IPython.display import display
    print(f"=== User {user_id} 최근 시청 {recent_n}개 ===")
    display(hist_df)

    print(f"\n=== User {user_id} 추천 Top-{topk} (SASRec) ===")
    display(rec_df)


In [30]:
# ---------------------------------------------------------
# 13. 임의의 유저 한 명에 대해
#     - 최근 시청 기록과
#     - SASRec 추천 Top-10 결과를 확인해보기
# ---------------------------------------------------------

# user_train 딕셔너리에서 유저 하나를 임의로 선택
#  - user_train: {user_id: [i1, i2, ..., iM]} 형태
#  - keys() 중 첫 번째 유저를 뽑아서 샘플로 사용
sample_user = list(user_train.keys())[0]

# 해당 유저에 대해:
#   - 최근에 본 영화 10개 (recent_n=10)
#   - 모델이 추천한 Top-10 영화 (topk=10)
# 를 영화 제목 기준으로 출력
show_user_history_and_recs(
    sample_user,
    topk=10,
    recent_n=10
)


=== User 1 최근 시청 10개 ===


,order,recent_watched
0,10,"Truth About Cats & Dogs, The (1996)"
1,9,Delicatessen (1991)
2,8,Kolya (1996)
3,7,"Grand Day Out, A (1992)"
4,6,Crumb (1994)
5,5,This Is Spinal Tap (1984)
6,4,Gattaca (1997)
7,3,"White Balloon, The (1995)"
8,2,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
9,1,Breaking the Waves (1996)



=== User 1 추천 Top-10 (SASRec) ===


,rank,recommended
0,1,Rosencrantz and Guildenstern Are Dead (1990)
1,2,Babe (1995)
2,3,To Kill a Mockingbird (1962)
3,4,Titanic (1997)
4,5,Stand by Me (1986)
5,6,Walkabout (1971)
6,7,Manhattan (1979)
7,8,Boogie Nights (1997)
8,9,Leaving Las Vegas (1995)
9,10,Kundun (1997)


### SASRec 예시 결과 (User 1)

#### 1) User 1의 최근 시청 이력 (최근 10편)

| 순서(최근순) | 최근 시청 영화 제목 |
|-------------|---------------------|
| 1 (가장 최근) | Truth About Cats & Dogs, The (1996) |
| 2 | Delicatessen (1991) |
| 3 | Kolya (1996) |
| 4 | Grand Day Out, A (1992) |
| 5 | Crumb (1994) |
| 6 | This Is Spinal Tap (1984) |
| 7 | Gattaca (1997) |
| 8 | White Balloon, The (1995) |
| 9 | Shanghai Triad (Yao a yao yao dao waipo qiao) ... |
| 10 | Breaking the Waves (1996) |

**해석**

- 전반적으로 **예술/작가주의 + 유럽/외국 영화** 비중이 높은 유저  
  - *Breaking the Waves, Kolya, Delicatessen, Shanghai Triad, White Balloon* 등  
- 동시에 **블랙 코미디/컬트 감성**이 강하게 드러남  
  - *Crumb, This Is Spinal Tap, Delicatessen* 같은 작품들  
- 여기에  
  - 철학적 SF 드라마: *Gattaca*  
  - 로맨틱 코미디: *Truth About Cats & Dogs*  
  도 섞여 있어, “진지한 드라마와 기괴한 유머를 둘 다 즐기는” 패턴으로 해석 가능  
- SASRec 입장에서는 위 시퀀스가 “이 유저의 **최근 취향 흐름**”을 나타내는 입력이 됨  

---

#### 2) SASRec가 예측한 User 1 추천 Top-10

| 순위 | 추천 영화 제목 |
|------|----------------|
| 1 | Rosencrantz and Guildenstern Are Dead (1990) |
| 2 | Flirting With Disaster (1996) |
| 3 | Heathers (1989) |
| 4 | Babe (1995) |
| 5 | Better Off Dead... (1985) |
| 6 | Strictly Ballroom (1992) |
| 7 | Adventures of Priscilla, Queen of the Desert, ... |
| 8 | Stand by Me (1986) |
| 9 | Muriel's Wedding (1994) |
| 10 | Clueless (1995) |

**해석**

- **톤·분위기 맞추기**  
  - *Rosencrantz and Guildenstern Are Dead, Heathers, Better Off Dead..., Flirting With Disaster*  
    → 블랙 코미디, 위트 있는 청춘/관계극 중심  
  - 이는 *Delicatessen, This Is Spinal Tap, Crumb*에서 보인 **기괴한 유머 + 인디 감성**과 자연스럽게 이어짐  
- **성장/관계 중심 드라마·코미디**  
  - *Stand by Me, Clueless, Muriel's Wedding, Babe, Strictly Ballroom*  
    → 성장, 정체성, 관계 변화에 초점을 둔 80~90년대 작품들  
  - *Breaking the Waves, Kolya, White Balloon*처럼 감정선이 강한 드라마 취향과도 맞닿아 있음  
- **국적·문화권 다양성 유지**  
  - *Muriel's Wedding, Strictly Ballroom, Adventures of Priscilla…* → 호주 영화  
  - 최근 히스토리에서 보였던 **유럽·외국 영화 선호**를 유지하면서,  
    영미권 인디/청춘 영화까지 함께 추천하고 있음  

---

#### 3) 이 예시가 보여주는 **SASRec의 특징**

- **시퀀스(순서)를 활용하는 모델**  
  - 단순히 “이 유저가 본 영화 전체 집합”이 아니라,  
    **어떤 분위기의 작품들을 어떤 순서로 봤는지**를 입력으로 사용  
  - 예: 최근에 예술·인디·블랙코미디 비중이 높아지면, 추천도 그 톤을 따라감  

- **최근 히스토리에 더 민감**  
  - 유저의 아주 오래된 시청 기록보다,  
    **최근에 연속으로 본 영화들의 분위기와 무드**를 더 강하게 반영  
  - 그래서 예술/드라마 위주의 최근 시청 이력 안에 섞여 있는  
    블랙코미디·성장물 취향을 포착해  
    *Heathers, Better Off Dead..., Stand by Me, Muriel's Wedding, Clueless* 같은 작품들도 함께 추천  

- **LightGCN 같은 정적 모델과의 대비 포인트**  
  - LightGCN: “본 적 있다 / 없다”라는 정적 관계 위주 →  
    시간 순서를 바꿔도 추천이 크게 안 변하는 경우가 많음  
  - SASRec: prefix(히스토리 구간)를 어디까지 주느냐에 따라 **추천 랭킹이 크게 바뀜**  
    → “시간에 따라 변하는 취향”과 “최근 무드”를 반영하는 **시퀀스 추천 모델**이라는 점을 설명하는 좋은 사례

In [31]:
# ---------------------------------------------------------
# 14. "prefix를 어떻게 자르느냐에 따라 추천이 어떻게 달라지는지" 비교
#    - sasrec_recommend_with_prefix:
#        → 임의의 시퀀스(prefix)를 넣고 Top-K 추천을 얻는 함수
#    - compare_prefix_effect:
#        → 같은 유저에 대해
#           (a) 초반 일부 prefix만 썼을 때의 추천
#           (b) 전체 prefix를 썼을 때의 추천
#          을 나란히 비교해서,
#          "SASRec가 시퀀스 길이에 민감하다"는 점을 보여주는 데 사용
# ---------------------------------------------------------

def sasrec_recommend_with_prefix(
    model,
    seq,
    topk=10,
    max_len=MAX_SEQ_LEN,
    device="cuda"
):
    """
    임의의 시퀀스(seq)를 직접 넣어서, SASRec 추천 Top-K를 얻는 함수.

    - user_id 없이, "아이템 시퀀스"만 가지고도 추천을 보고 싶을 때 사용.
      (예: 같은 유저의 prefix를 잘라서 여러 버전으로 넣어보는 경우)
    """
    model.eval() # 힌트 : 평가 모드
    with torch.no_grad():
        # 1) 입력 시퀀스 복사
        s = seq.copy()

        # 2) 시퀀스가 너무 길면 최근 max_len개만 사용
        if len(s) > max_len:
            s = s[-max_len:]

        # 3) 왼쪽 padding으로 길이를 max_len에 맞추기
        pad_len = max_len - len(s)
        s = [0] * pad_len + s

        # 4) Tensor로 변환 후 모델에 넣기
        seq_tensor = torch.LongTensor(s).unsqueeze(0).to(device)  # (1, L)
        logits = model.predict_next(seq_tensor)                   # (1, n_items+1)
        scores = logits.squeeze(0).cpu().numpy()                  # (n_items+1,)

        # padding id(0)는 추천 후보에서 제외
        scores[0] = -1e9

        # 점수 기준 내림차순 정렬 → 상위 topk개 선택
        ranking = np.argsort(-scores)
        top_items = ranking[:topk]

    return top_items


def compare_prefix_effect(user_id, topk=10, cut_ratio=0.5):
    """
    같은 유저에 대해 "prefix 길이"에 따른 추천 결과 차이를 비교하는 함수.

    - full_seq: 이 유저의 전체 시퀀스
    - early_prefix: full_seq의 앞부분 일부 (cut_ratio 비율만큼)
    - full_prefix : full_seq 전체

    → early_prefix로 추천한 Top-K vs full_prefix로 추천한 Top-K를
      한 테이블에서 비교해서 보여줌.
    """
    # 이 유저의 전체 시퀀스
    full_seq = user_train[user_id]

    # cut_ratio 만큼 앞에서 자른 위치 (최소 1 이상)
    # ex) cut_ratio=0.5고 full_seq 길이=100이면 cut_idx=50 → 앞 50개만 사용
    cut_idx = max(1, int(len(full_seq) * cut_ratio))

    early_prefix = full_seq[:cut_idx]   # 초반 일부 시퀀스
    full_prefix  = full_seq             # 전체 시퀀스

    # 1) 초반 prefix만 썼을 때의 추천
    early_recs = sasrec_recommend_with_prefix(
        model,
        early_prefix,
        topk=topk
    )

    # 2) 전체 prefix를 썼을 때의 추천
    full_recs = sasrec_recommend_with_prefix(
        model,
        full_prefix,
        topk=topk
    )

    # 3) 아이템 id → 영화 제목으로 변환
    early_titles = [id2title.get(i, f"item_{i}") for i in early_recs]
    full_titles  = [id2title.get(i, f"item_{i}") for i in full_recs]

    # 4) 두 결과를 나란히 보여줄 DataFrame 구성
    diff_df = pd.DataFrame({
        "rank": range(1, topk + 1),
        "early_prefix_recs": early_titles,
        "full_prefix_recs": full_titles
    })

    from IPython.display import display

    # -------------------------
    # 출력 부분
    # -------------------------
    print(f"=== User {user_id} prefix 비교 ===")
    print(f"- early prefix length: {len(early_prefix)}")
    print(f"- full  prefix length: {len(full_prefix)}\n")

    # 이 유저의 최근 시청 히스토리 일부도 같이 보여줌
    print("최근 히스토리 일부:")
    recent_titles = [
        id2title.get(i, f"item_{i}")
        for i in full_seq[-10:]
    ]
    display(pd.DataFrame({
        "order": list(range(len(recent_titles), 0, -1)),
        "recent_watched": list(reversed(recent_titles))
    }))

    # 초반 prefix vs 전체 prefix 추천 리스트 비교
    print("\n추천 Top-{} 비교 (초반 히스토리 vs 전체 히스토리)".format(topk))
    display(diff_df)


In [32]:
# ---------------------------------------------------------
# 15. 임의의 유저에 대해 "prefix 효과" 실제로 확인하기
#    - 같은 유저에 대해:
#        · 초반 40% 시청 이력만 사용했을 때 추천 Top-10
#        · 전체 시청 이력을 다 사용했을 때 추천 Top-10
#      을 비교해서,
#      "SASRec가 시퀀스 길이/최근 취향 변화에 민감하다"는 걸 눈으로 확인하는 예제.
# ---------------------------------------------------------

# user_train 딕셔너리에서 11번째 유저를 샘플로 선택
#  - 순서는 특별한 의미는 없고, 그냥 "아무 유저나" 하나 고른 것
sample_user = list(user_train.keys())[10]

# 이 유저에 대해 prefix 효과 비교:
#   - cut_ratio=0.4  → 전체 시퀀스 길이의 40%까지만 초반 prefix로 사용
#   - topk=10        → Top-10 추천 결과를 비교
compare_prefix_effect(
    sample_user,
    topk=10,
    cut_ratio=0.4
)


=== User 11 prefix 비교 ===
- early prefix length: 62
- full  prefix length: 156

최근 히스토리 일부:


,order,recent_watched
0,10,Benny & Joon (1993)
1,9,Pretty Woman (1990)
2,8,Shadowlands (1993)
3,7,"Room with a View, A (1986)"
4,6,Enchanted April (1991)
5,5,"Last of the Mohicans, The (1992)"
6,4,"Client, The (1994)"
7,3,Star Trek: The Motion Picture (1979)
8,2,Body Snatchers (1993)
9,1,Outbreak (1995)



추천 Top-10 비교 (초반 히스토리 vs 전체 히스토리)


,rank,early_prefix_recs,full_prefix_recs
0,1,Immortal Beloved (1994),Benny & Joon (1993)
1,2,Quiz Show (1994),Grease (1978)
2,3,"Remains of the Day, The (1993)",Dave (1993)
3,4,Dances with Wolves (1990),Sleepless in Seattle (1993)
4,5,Fried Green Tomatoes (1991),Somewhere in Time (1980)
5,6,Philadelphia (1993),Pretty Woman (1990)
6,7,Nobody's Fool (1994),"Age of Innocence, The (1993)"
7,8,My Left Foot (1989),It Could Happen to You (1994)
8,9,Field of Dreams (1989),Persuasion (1995)
9,10,Four Weddings and a Funeral (1994),Ghost (1990)


### SASRec Prefix 비교 예시 (User 11)

#### 1) Prefix 길이 정보

- early prefix length: **62**  
- full  prefix length: **156**

즉,  
- **early prefix** = 이 유저의 **초반 40% 정도 시청 이력만 본 경우**  
- **full prefix** = 전체 156편의 시청 이력을 **끝까지 반영한 경우**  
에 대해 각각 추천 Top-10을 비교한 결과입니다.

---

#### 2) User 11의 최근 시청 이력 (최근 10편)

| 순서(최근순) | 최근 시청 영화 제목 |
|-------------|---------------------|
| 1 (가장 최근) | Benny & Joon (1993) |
| 2 | Pretty Woman (1990) |
| 3 | Shadowlands (1993) |
| 4 | Room with a View, A (1986) |
| 5 | Enchanted April (1991) |
| 6 | Last of the Mohicans, The (1992) |
| 7 | Client, The (1994) |
| 8 | Star Trek: The Motion Picture (1979) |
| 9 | Body Snatchers (1993) |
| 10 | Outbreak (1995) |

**대략적인 분위기**

- 직전 시청 기록을 보면, 전반적으로 **로맨스/멜로/관계 중심 드라마** 비중이 매우 큼  
  - *Benny & Joon, Pretty Woman, Shadowlands, A Room with a View, Enchanted April* 등  
- 여기에  
  - 역사/전쟁·서사: *Last of the Mohicans*  
  - 법정 스릴러: *The Client*  
  - SF / 호러 / 재난: *Star Trek: The Motion Picture, Body Snatchers, Outbreak*  
  도 포함되어 있어 “관계·감정선을 좋아하지만 장르도 가끔 섞어 보는 유저”로 볼 수 있음.  
- 특히 최근 몇 편은 **관계·로맨스·정서적인 드라마** 쪽으로 몰려 있는 패턴이 뚜렷함 →  
  SASRec 입장에서는 “최근 취향이 로맨스/드라마 쪽으로 이동 중”이라고 읽을 수 있음.

---

#### 3) 추천 Top-10 비교 (초반 히스토리 vs 전체 히스토리)

| 순위 | early prefix 기준 추천 Top-10 | full prefix 기준 추천 Top-10 |
|------|-------------------------------|--------------------------------|
| 1 | Immortal Beloved (1994) | Benny & Joon (1993) |
| 2 | Quiz Show (1994) | Ghost (1990) |
| 3 | Remains of the Day, The (1993) | It Could Happen to You (1994) |
| 4 | What's Eating Gilbert Grape (1993) | Age of Innocence, The (1993) |
| 5 | Four Weddings and a Funeral (1994) | Grease (1978) |
| 6 | Fried Green Tomatoes (1991) | Don Juan DeMarco (1995) |
| 7 | Piano, The (1993) | Pretty Woman (1990) |
| 8 | Little Women (1994) | Shadowlands (1993) |
| 9 | Philadelphia (1993) | Sleepless in Seattle (1993) |
| 10 | Rudy (1993) | Dirty Dancing (1987) |

---

#### 4) 어떤 변화가 일어났는가?

##### (1) Early prefix (초반 62편만 봤을 때)

- 추천 리스트 키워드:
  - **품격 있는 드라마·문학 원작 계열**  
    - *The Remains of the Day, The Piano, Little Women, Philadelphia, Immortal Beloved*  
  - **관계·성장·정체성 중심 드라마/로맨스**  
    - *What's Eating Gilbert Grape, Four Weddings and a Funeral, Fried Green Tomatoes, Rudy*  
- 해석:
  - early prefix만 보면 모델은 이 유저를  
    > “90년대 평단 호평 드라마/문학 원작/성장 드라마를 폭넓게 보는 사람”  
    정도로 이해하고, 비교적 **진지한 드라마·명작** 쪽 영화들을 상위에 추천하고 있음.  
  - 로맨스도 있지만, 전반적으로 “품격 있는 드라마/문학 영화”에 약간 더 무게가 실린 느낌.

##### (2) Full prefix (전체 156편을 다 봤을 때)

- 추천 리스트 양상이 훨씬 명확하게 **로맨스/관계 영화**로 쏠림:
  - 순수 로맨스/로코/댄스 로맨스:
    - *Ghost, It Could Happen to You, Pretty Woman, Sleepless in Seattle, Dirty Dancing, Don Juan DeMarco, Grease*  
  - 시대/계급/연애가 섞인 드라마:
    - *The Age of Innocence, Benny & Joon, Shadowlands*  
- 특히 눈에 띄는 점:
  - **최근에 실제로 본 영화들이 그대로 추천 상위권에 등장**  
    - *Benny & Joon*, *Pretty Woman*, *Shadowlands* 등  
  - 전체 이력을 보면서,  
    “결국 이 유저는 로맨스·관계 중심 영화를 반복해서 보는 경향”을 강하게 포착한 것으로 볼 수 있음.
- 해석:
  - full prefix까지 반영하면, 모델의 관점은  
    > “드라마도 좋아하지만, 특히 **로맨스/관계 중심 영화**를 오래·꾸준히 즐겨 보는 유저”  
    쪽으로 업데이트된다.  
  - 그 결과, 추천 Top-10이 **거의 전부 로맨스/로코/관계 영화**로 재편됨.

---

#### 5) 이 예시가 보여주는 **SASRec의 “시퀀스 모델” 특성**

1. **히스토리의 “순서”와 “최신 부분”에 민감하다**  
   - 같은 유저라도  
     - 초반 62편만 반영할 때 → “품격 있는 드라마/문학 영화 + 성장 드라마” 중심 추천  
     - 전체 156편을 반영할 때 → “로맨스/관계 중심 영화”가 압도적으로 많은 추천  
   - 즉, SASRec는 단순 장르 통계가 아니라  
     **시간이 흐르면서 취향이 어느 방향으로 수렴하는지**까지 본다.

2. **최근 히스토리와 누적 패턴이 추천을 “재구성”한다**  
   - early prefix에서는 드라마/문학 계열과 성장물 비중이 높았지만,  
   - full prefix에서는  
     - 실제로 가장 최근에 본 영화들(*Benny & Joon, Pretty Woman, Shadowlands*)과  
     - 과거부터 꾸준히 즐겨온 로맨스 계열 영화들이  
       **추천 상위권으로 밀려 올라온다.**  
   - 이는 “최근 + 누적” 정보가 함께 작용해  
     **유저의 장기 취향 중심축(=로맨스/관계 영화)**을 더 명확히 반영했기 때문으로 볼 수 있음.

3. **정적 모델과의 대비 포인트**  
   - LightGCN처럼 “어떤 아이템을 봤는가”만 쓰는 정적 CF 모델은,  
     prefix 길이를 조금 바꿔도 추천 Top-10이 크게 안 바뀌는 경우가 많다.  
   - 하지만 SASRec에서는  
     - early vs full prefix만 바꿔도 추천 리스트가  
       “드라마/명작 중심 → 로맨스/관계 중심”으로 눈에 띄게 재구성됨.  
   - 이 사례는 SASRec가 **시퀀스(Sequential) 정보**를 활용해  
     “시간에 따라 변하거나 더 선명해지는 취향”을 포착하는 모델이라는 점을 보여주기에 좋다.